# Make DETECTION and LATENCY using surpdata, sigdata.pckl

We detet significant surprises for every neuron and save them

It uses surprises from 5 to 55ms since after 55ms overlaps with next stimulus response
It uses the Blanksurprise data for the thresholds

It is made for:
    - every neuron
    - every whisker
    - both directions
        
Three main cells used in order do:
- Compute surprise
- Detect significant whiskers
- Compute Latencies

Parameters are:
    - sizesaboveth
    - consecaboveth

For latencies we remind:
    - maxbin 20
    - surpmax 55

In [27]:
%pylab inline
#%autoreload 1
#%aimport surp_functions

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [28]:
from pylab import *
from scipy import *
import numpy as np
from attrdict import AttrDict
from scipy.stats import poisson

import sys
sys.path.append('../functions') #this is where we put all the functions.py
import exps_files_folders as EFF
Expe, Vtags, Stim, rootF, stimFolder = EFF.eff()
from save_load import *

from surp_functions import*

# Make Detection

In [34]:
global binname, textname
#---------------------------------------------------------------------------------------
SelExp = [20,22,23,24,26,27,28,29,30,31,32] #Expe                           #select experiment numbers!

Folder = '/home/matias/WORKSPACE/S2_git/data'    

thfile = Folder + '/Blanksurprise'
Th = load_obj(thfile)

# Threshold was computed for this 5 pvals: (0.01, 0.005, 0.001, 0.0005, 0.0001)
# we select HERE the 0.005 at 4th position
position = 5

pvalue = Th.pval_list[position-1]
print('Selected pval for blank threshold: ', pvalue)

thresholds = Th.surp_thresh[:,position-1]
print('Thresholds for all binsizes :')
print( np.reshape(thresholds,[-1,1]))

#--------------------------------------------------------------------------------
# Loop Experiments
#--------------------------------------------------------------------------------
for expe in SelExp:
    print(expe)
    
    Detection = {}
    #--------------------
    # To load Surprise
    surpdata = {}
    surpfile = Folder +'/surpdata' + str(expe)
    surpdata = load_obj(surpfile)
    #--------------------
       
    idx = sorted(list(surpdata.keys()))
    #--------------------------------------------------------------------------------
    # loop neurons
    #--------------------------------------------------------------------------------
    for n in idx[:]:
        #print('   neuron: ' , n)
        clus = surpdata[n].clusnum
        meas = surpdata[n].meas
        shank = surpdata[n].shank
        surp = surpdata[n].surprise_data.Data
       
        #-------------------------------------------------------    
        Detection[n] = AttrDict({})
        Detection[n].update({'clusnum': clus, 'exp' : int(expe) , 'meas': meas , 'shank': shank})
        #-------------------------------------------------------

        # we give the whisker number blank stimulus
        wblank = 20   
        if expe <= 22:     #Only for EXP 22 or less: odd ELPHY piezo assignement
            wblank = 11
        #-------------------------------------------------------
        #----------------------------------------------------
        # we detect significant whiskers
        surpmin = 5       #minimum latency in ms, so significance is computed from here
        
        nconsecaboveth = 1   # how many bins consecutive with surp above th to call it significant
        nsizesaboveth = 2    #number of binnings where surprise to be significative for whisker to be significative

        Detection[n].update({'detection_data': BuildSig(surp,thresholds,wblank,nconsecaboveth,surpmin,nsizesaboveth)})
                               
        #----------------------------------------------------
        #-------------------------------------------------------
        # we get the Latencies, only for the significant whiskers
        surpmax= 55
        maxbin = 20
        Latencies = GetLatency(surp,Detection[n].detection_data, surpmax, maxbin, surpmin, thresholds)
        Detection[n].update({'latency_data' : Latencies })
       
    print('Saving')
    filesave =Folder +'/sigdata' + str(expe)
    save_obj(Detection,filesave)                 
 

Selected pval for blank threshold:  0.0001
Thresholds for all binsizes :
[[ 4.7]
 [ 4.3]
 [ 4.4]
 [ 4.4]
 [ 4.4]
 [ 4.5]
 [ 4.8]
 [ 5. ]
 [ 5.2]
 [ 5.4]
 [ 5.1]
 [ 5.2]
 [ 6.8]
 [ 6.8]
 [ 6.8]
 [ 6.8]
 [ 6.8]
 [ 6.8]
 [ 6.8]
 [ 6.8]]
20
Saving
22
Saving
23
Saving
24
Saving
26
Saving
27
Saving
28
Saving
29
Saving
30
Saving
31
Saving
32
Saving


In [35]:
Folder = '/home/matias/WORKSPACE/S2_git/data'    
filesave =Folder +'/sigdata20' 

a = load_obj(filesave)

idx = sorted(list(a.keys()))

i=0

n = idx[i]
print(n)
#print(a[n].keys())
print(a[n].detection_data.keys())
print(a[n].detection_data.Sig.reshape([5,5,2]))
a[n].detection_data.Sig_strength_norm.reshape([5,5,2])

exp20_m1s1_c29
dict_keys(['Sig_strength', 'PWstrong', 'logic_tree_significants', 'PW', 'Sig_top', 'Sig_strength_norm', 'Sig_sizes', 'logic_tree_sig_sizes', 'Sig'])
[[[ 0.  1.]
  [ 0.  0.]
  [ 1.  1.]
  [ 1.  1.]
  [ 0.  0.]]

 [[ 1.  1.]
  [ 1.  1.]
  [ 0.  1.]
  [ 0.  0.]
  [ 1.  1.]]

 [[ 1.  1.]
  [ 0.  0.]
  [ 0.  1.]
  [ 0.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 1.  0.]
  [ 0.  0.]
  [ 1.  0.]
  [ 1.  0.]]

 [[ 0.  0.]
  [ 0.  1.]
  [ 0.  1.]
  [ 1.  0.]
  [ 1.  1.]]]


array([[[ 0.        ,  0.03053435],
        [ 0.        ,  0.        ],
        [ 0.13282443,  0.05343511],
        [ 0.08396947,  0.00763359],
        [ 0.        ,  0.        ]],

       [[ 0.78473282,  0.80916031],
        [ 0.64885496,  0.50839695],
        [ 0.        ,  0.03206107],
        [ 0.        ,  0.        ],
        [ 0.01526718,  0.01526718]],

       [[ 0.66870229,  0.38625954],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.03664122],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.01374046]],

       [[ 0.04732824,  0.        ],
        [ 0.0519084 ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.0778626 ,  0.        ],
        [ 0.01068702,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.00916031],
        [ 0.        ,  0.02900763],
        [ 0.18931298,  0.        ],
        [ 1.        ,  0.99083969]]])

In [3]:
Folder = '/home/matias/WORKSPACE/S2_git/data'    


filesave =Folder +'/sigdata20' 

a = load_obj(filesave)

idx = list(a.keys())

n = idx[0]


a[n].keys()

dict_keys(['meas', 'clusnum', 'detection_data', 'exp', 'shank'])